In [21]:
import pandas as pd
import re as r
import numpy as np

import pandas as pd

from keras.models import Sequential

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.utils.np_utils import to_categorical


In [22]:
df = pd.read_csv("final.csv")

In [23]:
df.head()

,comment_text,pol
0,logan paul big day,0.0
1,follow start vine channel seen 365 vlog,0.0
2,say hi kong maverick,0.0
3,fan attend,0.0
4,trend,0.0


In [24]:
df.isna().sum()

comment_text    137
pol               0
dtype: int64

In [25]:
df.count()

comment_text     9863
pol             10000
dtype: int64

In [26]:
df = df.dropna(how='any',axis=0) 

In [27]:
df.count()

comment_text    9863
pol             9863
dtype: int64

In [28]:
df.pol.value_counts()

 1.0    4172
 0.0    3956
-1.0    1735
Name: pol, dtype: int64

In [29]:
df =df[:2000]

In [30]:
df.count()

comment_text    2000
pol             2000
dtype: int64

In [31]:
df['comment_text'] = df['comment_text'].str.lower()

In [32]:
df['comment_text'] = df['comment_text'].apply((lambda x: r.sub('[^a-zA-z0-9\s]','',x)))

In [33]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['comment_text'].values)
X = tokenizer.texts_to_sequences(df['comment_text'].values)

In [34]:
X = pad_sequences(X)
X.shape[1]

1243

In [35]:
X = pad_sequences(X,maxlen=1250)

In [36]:
X.shape

(2000, 1250)

In [37]:
Y = df['pol']

In [38]:
Y.shape

(2000,)

In [39]:
embed_dim = 500
lstm_out = 6

model = Sequential()
model.add(Embedding(156, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1250, 500)         78000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1250, 500)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 6)                 12168     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 90,175
Trainable params: 90,175
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

In [41]:
model.fit(X_train, y_train,validation_data = (X_test,y_test),epochs = 9, batch_size=64)

Epoch 1/9


InvalidArgumentError:  indices[62,1245] = 385 is not in [0, 156)
	 [[node sequential_1/embedding_1/embedding_lookup (defined at <ipython-input-41-2e1bf1f14c0b>:1) ]] [Op:__inference_train_function_4690]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
 sequential_1/embedding_1/embedding_lookup/2974 (defined at C:\Users\kammari.vara.prasad\AppData\Local\Continuum\anaconda3\lib\contextlib.py:112)

Function call stack:
train_function


In [ ]:
model.evaluate(X_test,y_test)